In [111]:
from sqlglot import parse_one, exp
from sqlglot.dialects.tsql import TSQL
import pypyodbc as odbc
odbc.lowercase = False
import configparser
import copy
from collections import defaultdict
from collections import OrderedDict 
import pandas as pd
import pypyodbc as odbc
import configparser
import os
import json
import re
from parse_lineages import *
from parse_nodes import *

odbc.lowercase = False

In [112]:
nodes = pd.read_csv('output-data/nodes.csv')
nodes.head()

,Unnamed: 0,NAME_NODE,LABEL_NODE,WHERE,FUNCTION,ON,COLOR,ID
0,0,subquery1_5,json_data0@subquery1_5,orders.status = 'Completed',subquery,NaN,grey,0
1,1,orders,orders,NaN,table,NaN,grey,1
2,2,subquery1_4,json_data0@subquery1_4,NaN,subquery,NaN,grey,2
3,3,reviews,reviews,NaN,table,NaN,grey,3
4,4,subquery1_3,json_data0@subquery1_3,NaN,subquery,['order_items.product_id = products.product_id...,grey,4


In [113]:
queries = [] # queries content list
files = [] # file names list
for file in os.listdir("gpt-sql-queries-extracted3"):
    files.append(file.split('.')[0])
    with open(f'gpt-sql-queries-extracted3/{file}', 'r') as file:
        data = json.load(file)

    queries.append(data)

print(files[0])
queries[0]

json_data0


{'modified_SQL_query': 'CREATE VIEW CUSTOMER_ORDER AS SELECT c.customer_id, c.first_name, c.last_name, c.email, recent_orders.order_id, recent_orders.order_date, recent_orders.total_order_amount, recent_payments.total_payment_amount, recent_order_items.product_name, recent_order_items.category_name, recent_order_items.quantity, recent_order_items.unit_price, recent_order_items.total_item_amount, CAST(avg_product_rating.avg_rating AS DECIMAL(3,2)) AS avg_product_rating, CAST(total_customer_spend.total_spent AS DECIMAL(10,2)) AS total_customer_spent FROM customers c JOIN subquery1_1 recent_orders ON c.customer_id = recent_orders.customer_id JOIN subquery1_2 recent_payments ON recent_orders.order_id = recent_payments.order_id JOIN subquery1_3 recent_order_items ON recent_orders.order_id = recent_order_items.order_id JOIN subquery1_4 avg_product_rating ON recent_order_items.product_id = avg_product_rating.product_id JOIN subquery1_5 total_customer_spend ON c.customer_id = total_customer_sp

In [114]:
def parse_table(table, table_alias_list, subquery=True):    
    #table = element.this.this
    #table_name = table.name
    #catalog =  table.catalog
    #db = table.db

    if subquery == False:
        table_alias =  table.alias
        table_name = table.name
        table_db = table.db
        table_catalog = table.catalog

    else:
        table_alias = table.alias
        source = table.this.args["from"]
        table_name= source.this.name
        table_catalog =  source.this.catalog
        table_db = source.this.db
        
        
    if " " in table_name:
        table_name = table_name.replace(" ", "")

    if table_catalog == "":    
        result = (table_db+"."+table_name, table_alias)
    elif table_db == "":
        result = (table_catalog+"."+table_name, table_alias)
    else:
        result = (table_catalog+"."+ table_db+"."+table_name, table_alias)

    table_alias_list.append(result)
    return result

In [115]:
lineages_dfs = []
trees = []


for i, file in enumerate(os.listdir("gpt-sql-queries-extracted3")):
    filename = file.split('.')[0]
    print(filename)
    files.append(file.split('.')[0])
    with open(f'gpt-sql-queries-extracted3/{file}', 'r') as file:
        data = json.load(file)

    queries.append(data)

    # reverse subqueries dict to start from deepest level
    query_subqueries = dict(reversed(list(queries[i]['subquery_dictionary'].items())))
    query_subqueries['main_query'] = queries[i]['modified_SQL_query']
    query_subqueries

    lineages = [] # list of dictionaries with the nodes

    for name_query in query_subqueries:

        query = query_subqueries[name_query]

        if query.startswith("("):
            query = query.strip("()")
        else:
            pass

        ast = parse_query(query) # get parsed tree

        if 'subquery' in name_query: # if the query is a subquery then the name is the dict key, else the name is the target table
            target_node = name_query
            target_columns =[]

        else:
            target_columns =[]

            try: # try with create table
                target_node = list(ast.find_all(exp.Create))[0].this.this.this
            except IndexError: # else try with insert into table
                target_node = list(ast.find_all(exp.Insert))[0].this.this

                insert_obj = list(ast.find_all(exp.Insert))[0]
                target_columns = list(insert_obj.find_all(exp.Column))
                target_columns = [[i] for i in target_columns]
             

        space_table = find_table_w_spaces(ast) # list with tables with spaces (sqlglot cant parse them)

        space_table = list(set(space_table)) # a list of tuples with table names paired (space removed original - original ) Eg. (OrderDetails, Order Details)

        alias_table = get_tables(ast) # parse table name + table alias

        tree = replace_aliases(query) # transform query by removing table aliases

        if target_columns == []:
            select_statement, target_columns = extract_target_columns(tree) # extract target columns
        else:
            select_statement, x = extract_target_columns(tree) # extract target columns

        #print(target_columns[0])
        #print(target_columns)

        replaced_trees = [x.transform(transformer_functions) for x in select_statement] # replace columns aliases
        trees.append(replaced_trees)

        # add possible transformation to columns
        transformations = extract_transformation(replaced_trees)
        target_columns = list(zip(target_columns, transformations)) 

        print(target_node)
        
        lineages = extract_source_target_transformation(target_columns, lineages, space_table, target_node) # append lineages of node to list


    lineages = pd.DataFrame(lineages)

    lineages = lineages.explode('SOURCE_COLUMNS').reset_index()

    lineages['FILE_NAME'] = filename
    lineages['ROW_ID'] = 0#lineages.index()
    lineages['LINK_VALUE'] =1

    lineages['SOURCE_NODE'] = lineages['SOURCE_COLUMNS'].apply(lambda x:".".join(x.split('.')[0:-1]))
    lineages['TARGET_NODE'] = lineages['TARGET_COLUMN'].apply(lambda x:".".join(x.split('.')[0:-1]))

    lineages['SOURCE_FIELD'] = lineages['SOURCE_COLUMNS'].apply(lambda x:x.split('.')[-1])
    lineages['TARGET_FIELD'] = lineages['TARGET_COLUMN'].apply(lambda x:x.split('.')[-1])

    lineages['SOURCE_NODE'] = [f'{filename}@{i}' if 'subquery' in i else i for i in lineages['SOURCE_NODE'] ]
    lineages['TARGET_NODE'] = [f'{filename}@{i}' if 'subquery' in i else i for i in lineages['TARGET_NODE']]


    lineages['COLOR'] =  ["aliceblue" if i == "" else "orangered" for i in lineages['TRANSFORMATION']]

    # merge source id
    lineages = pd.merge(lineages, nodes[['ID', 'LABEL_NODE']], left_on='SOURCE_NODE', right_on = 'LABEL_NODE', how='left')
    lineages['SOURCE_NODE'] = lineages['ID']
    lineages.drop(columns=['ID', 'LABEL_NODE'], inplace=True)
    # merge target id
    lineages = pd.merge(lineages, nodes[['ID', 'LABEL_NODE']], left_on='TARGET_NODE', right_on = 'LABEL_NODE', how='left')
    lineages['TARGET_NODE'] = lineages['ID']
    lineages.drop(columns=['ID', 'LABEL_NODE'], inplace=True)

    lineages = lineages.drop_duplicates().reset_index(drop=True)


    lineages.to_csv(f"output-data/lineages/lineage-{filename}.csv")
    lineages_dfs.append(lineages)

json_data0
subquery1_5
subquery1_4
subquery1_3
subquery1_2
subquery1_1
CUSTOMER_ORDER
json_data1
subquery1_2
subquery1_1
CUSTOMER_BANK_DETAILS
json_data2
subquery2_1
subquery1_4
subquery1_3
subquery1_2
subquery1_1
INVESTOR_OVERVIEW
json_data3
subquery1_3
subquery1_2
subquery1_1
CUSTOMER_SUBSCRIPTION_DETAILS
json_data4
subquery1_4
subquery1_3
subquery1_2
subquery1_1
VENDOR_PERFORMANCE_ANALYSIS


In [116]:
lineages_dfs[0]

,index,SOURCE_COLUMNS,TARGET_COLUMN,TRANSFORMATION,FILE_NAME,ROW_ID,LINK_VALUE,SOURCE_NODE,TARGET_NODE,SOURCE_FIELD,TARGET_FIELD,COLOR
0,0,orders.customer_id,subquery1_5.customer_id,,json_data0,0,1,1.0,0,customer_id,customer_id,aliceblue
1,1,orders.total_amount,subquery1_5.total_spent,SUM(total_amount),json_data0,0,1,1.0,0,total_amount,total_spent,orangered
2,2,reviews.product_id,subquery1_4.product_id,,json_data0,0,1,3.0,2,product_id,product_id,aliceblue
3,3,reviews.rating,subquery1_4.avg_rating,AVG(rating),json_data0,0,1,3.0,2,rating,avg_rating,orangered
4,4,order_items.order_id,subquery1_3.order_id,,json_data0,0,1,5.0,4,order_id,order_id,aliceblue
5,5,products.product_name,subquery1_3.product_name,,json_data0,0,1,6.0,4,product_name,product_name,aliceblue
6,6,categories.category_name,subquery1_3.category_name,,json_data0,0,1,NaN,4,category_name,category_name,aliceblue
7,7,order_items.quantity,subquery1_3.quantity,,json_data0,0,1,5.0,4,quantity,quantity,aliceblue
8,8,order_items.unit_price,subquery1_3.unit_price,,json_data0,0,1,5.0,4,unit_price,unit_price,aliceblue
9,9,order_items.quantity,subquery1_3.total_item_amount,(quantity * unit_price),json_data0,0,1,5.0,4,quantity,total_item_amount,orangered


In [117]:
lineages_dfs.__len__()

5

In [118]:
lineages_dfs[0]['TRANSFORMATION'][30] 

# target node in case of insert into should also include the new columns
# when two columns are multiplied that does not count as transformation, fix that

'CAST(avg_rating AS NUMERIC(3, 2))'

In [119]:
['TRANSFORMATION','SOURCE_FIELD','SOURCE_NODE','TARGET_FIELD','TARGET_NODE','LINK_VALUE','ROW_ID','COLOR']
#0,,ROCANCEL,3,ROCANCEL,4,1,0,aliceblue


['TRANSFORMATION',
 'SOURCE_FIELD',
 'SOURCE_NODE',
 'TARGET_FIELD',
 'TARGET_NODE',
 'LINK_VALUE',
 'ROW_ID',
 'COLOR']

In [120]:
# parse limit 1
# fix source columns and target node when insert into


In [121]:
p

NameError: name 'p' is not defined

In [ ]:


def split_at_last_as(input_string): # function to split transformation string at last " AS "
    split_point = input_string.rfind(' AS ')
    if split_point == -1:
        return input_string, ''
    return input_string[:split_point], input_string[split_point + 4:]


def extract_source_target_transformation(target_columns, lineages, space_table, target_node_name):
    # extract source columns
    for target_column in target_columns:
        source_columns = []

        for source_column in target_column[0]:
            table = source_column.table
            catalog = source_column.catalog
            db = source_column.db
            column = source_column.name

            for w in space_table:
                if table == w[0]:
                    table = w[1]

            if catalog !="" and db !="":
                source_column_complete = catalog + "." +  db +"." + table +"." +column
                #matching = catalog + "." +  db +"." + table   

            elif catalog == "" and db == "":
                source_column_complete = table +"." +column
            elif catalog == "":    
                source_column_complete = db + "." + table + "."+column
            elif db == "":
                source_column_complete = catalog + "." + table +"." +column

            print(source_column_complete)

            source_columns.append(source_column_complete)
                
        if source_columns != []:
            if 'AS' in target_column[1]: # if there is an alias, append formula and alias
                lineages.append({'SOURCE_COLUMNS':source_columns, 'TARGET_COLUMN':f"{target_node_name}.{split_at_last_as(target_column[1])[1].strip()}", 'TRANSFORMATION':split_at_last_as(target_column[1])[0].strip()})

               # lineages.append({'source_columns':source_columns, 'target_column':f"{target_node_name}.{target_column[1].split(' AS ')[-1].strip()}", 'transformation':target_column[1].split(' AS ')[0].strip()})
            else:

                lineages.append({'SOURCE_COLUMNS':source_columns, 'TARGET_COLUMN':f'{target_node_name}.{source_columns[0].split(".")[-1]}', 'TRANSFORMATION': target_column[1]})
    return lineages


In [ ]:
lineages_dfs = []
trees = []


filename = 'json_data5'
print(filename)
with open(f'gpt-sql-queries-extracted2/json_data4.json', 'r') as file:
    data = json.load(file)

queries.append(data)

query_subqueries = dict(reversed(list(queries[i]['subquery_dictionary'].items())))
query_subqueries['main_query'] = queries[i]['modified_SQL_query']
query_subqueries

lineages = [] # list of dictionaries with the nodes

for name_query in query_subqueries:

    query = query_subqueries[name_query]

    if query.startswith("("):
        query = query.strip("()")
    else:
        pass

    ast = parse_query(query) # get parsed tree

    if 'subquery' in name_query: # if the query is a subquery then the name is the dict key, else the name is the target table
        target_node = name_query
        target_columns =[]

    else:
        target_columns =[]

        try: # try with create table
            target_node = list(ast.find_all(exp.Create))[0].this.this.this
        except IndexError: # else try with insert into table
            target_node = list(ast.find_all(exp.Insert))[0].this.this

            insert_obj = list(ast.find_all(exp.Insert))[0]
            target_columns = list(insert_obj.find_all(exp.Column))
            target_columns = [[i] for i in target_columns]
            

    space_table = find_table_w_spaces(ast) # list with tables with spaces (sqlglot cant parse them)

    space_table = list(set(space_table)) # a list of tuples with table names paired (space removed original - original ) Eg. (OrderDetails, Order Details)

    alias_table = get_tables(ast) # parse table name + table alias

    tree = replace_aliases(query) # transform query by removing table aliases
    

    if target_columns == []:
        select_statement, target_columns = extract_target_columns(tree) # extract target columns
    else:
        select_statement, x = extract_target_columns(tree) # extract target columns

    #print(target_columns[0])
    #print(target_columns)

    replaced_trees = [x.transform(transformer_functions) for x in select_statement] # replace columns aliases
    trees.append(replaced_trees)

    # add possible transformation to columns
    transformations = extract_transformation(replaced_trees)
    target_columns = list(zip(target_columns, transformations)) 

    
    lineages = extract_source_target_transformation(target_columns, lineages, space_table, target_node) # append lineages of node to list


lineages = pd.DataFrame(lineages)

lineages = lineages.explode('SOURCE_COLUMNS').reset_index()

lineages['FILE_NAME'] = filename
lineages['ROW_ID'] = 0#lineages.index()
lineages['LINK_VALUE'] =1

lineages['SOURCE_NODE'] = lineages['SOURCE_COLUMNS'].apply(lambda x:".".join(x.split('.')[0:-1]))
lineages['TARGET_NODE'] = lineages['TARGET_COLUMN'].apply(lambda x:".".join(x.split('.')[0:-1]))

lineages['SOURCE_FIELD'] = lineages['SOURCE_COLUMNS'].apply(lambda x:x.split('.')[-1])
lineages['TARGET_FIELD'] = lineages['TARGET_COLUMN'].apply(lambda x:x.split('.')[-1])

lineages['SOURCE_NODE'] = [f'{filename}@{i}' if 'subquery' in i else i for i in lineages['SOURCE_NODE'] ]
lineages['TARGET_NODE'] = [f'{filename}@{i}' if 'subquery' in i else i for i in lineages['TARGET_NODE']]


lineages['COLOR'] =  ["aliceblue" if i == "" else "orangered" for i in lineages['TRANSFORMATION']]

# merge source id
#lineages = pd.merge(lineages, nodes[['ID', 'LABEL_NODE']], left_on='SOURCE_NODE', right_on = 'LABEL_NODE', how='left')
#lineages['SOURCE_NODE'] = lineages['ID']
#lineages.drop(columns=['ID', 'LABEL_NODE'], inplace=True)
## merge target id
#lineages = pd.merge(lineages, nodes[['ID', 'LABEL_NODE']], left_on='TARGET_NODE', right_on = 'LABEL_NODE', how='left')
#lineages['TARGET_NODE'] = lineages['ID']
#lineages.drop(columns=['ID', 'LABEL_NODE'], inplace=True)

lineages

json_data5
SOURCE_DB.investors.investor_id
SOURCE_DB.investors.investor_id
SOURCE_DB.investors.investor_id
SOURCE_DB.investors.investor_id
s.stock_id
s.ticker
s.company_name
s.sector
.subquery1_1
.subquery1_2
.subquery1_3
.subquery1_4
s.stock_id
s.ticker
s.company_name
s.sector
s.stock_id
t.stock_id
t.portfolio_id
p.portfolio_id
p.investor_id
i.investor_id
s.stock_id
d.stock_id
s.stock_id


,index,SOURCE_COLUMNS,TARGET_COLUMN,TRANSFORMATION,FILE_NAME,ROW_ID,LINK_VALUE,SOURCE_NODE,TARGET_NODE,SOURCE_FIELD,TARGET_FIELD,COLOR
0,0,SOURCE_DB.investors.investor_id,subquery1_4.investor_id,,json_data5,0,1,SOURCE_DB.investors,json_data5@subquery1_4,investor_id,investor_id,aliceblue
1,1,SOURCE_DB.investors.investor_id,subquery1_3.investor_id,,json_data5,0,1,SOURCE_DB.investors,json_data5@subquery1_3,investor_id,investor_id,aliceblue
2,2,SOURCE_DB.investors.investor_id,subquery1_2.investor_id,,json_data5,0,1,SOURCE_DB.investors,json_data5@subquery1_2,investor_id,investor_id,aliceblue
3,3,SOURCE_DB.investors.investor_id,subquery1_1.investor_id,,json_data5,0,1,SOURCE_DB.investors,json_data5@subquery1_1,investor_id,investor_id,aliceblue
4,4,s.stock_id,PROCESS_DB.stock_performance.stock_id,,json_data5,0,1,s,PROCESS_DB.stock_performance,stock_id,stock_id,aliceblue
5,5,s.ticker,PROCESS_DB.stock_performance.ticker,,json_data5,0,1,s,PROCESS_DB.stock_performance,ticker,ticker,aliceblue
6,6,s.company_name,PROCESS_DB.stock_performance.company_name,,json_data5,0,1,s,PROCESS_DB.stock_performance,company_name,company_name,aliceblue
7,7,s.sector,PROCESS_DB.stock_performance.sector,,json_data5,0,1,s,PROCESS_DB.stock_performance,sector,sector,aliceblue
8,8,.subquery1_1,PROCESS_DB.stock_performance.total_trades_last...,COUNT(trade_id) FILTER(WHERE trade_date >= DAT...,json_data5,0,1,,PROCESS_DB.stock_performance,subquery1_1,total_trades_last_month,orangered
9,9,.subquery1_2,PROCESS_DB.stock_performance.total_trade_value...,SUM(quantity * price_per_share) FILTER(WHERE t...,json_data5,0,1,,PROCESS_DB.stock_performance,subquery1_2,total_trade_value_last_month,orangered
